In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import cv2
import os

# specify GPU
device = torch.device("cuda")

In [6]:
cd = pd.read_excel('data/results_large.xlsx', engine = "openpyxl")
file_list = list(cd['file'])

In [9]:
from os.path import exists

image1 = {}
image2 = {}
i = 0

A = {}
L = {}
l = {}
w = {}
h = {}
for item in file_list:
    path1 = os.path.join('data/depth', item+'.obj.png')
    path2 = os.path.join('data/normal', item+'.obj.png')
#     path = os.path.join('data/perspective', item+'.obj.png')
#     path = os.path.join('data/', item+".obj", 'top.png')
#     path = os.path.join('data/', item+".obj", 'bottom.png')
#     path = os.path.join('data/', item+".obj", 'left.png')
#     path = os.path.join('data/', item+".obj", 'right.png')
#     path = os.path.join('data/', item+".obj", 'front.png')
#     path = os.path.join('data/', item+".obj", 'back.png')
    if exists(path1):
        i += 1
        img = cv2.imread(path1)
        img = cv2.resize(img, (384,384), interpolation = cv2.INTER_AREA)
        image1[item] = img
        img = cv2.imread(path2)
        img = cv2.resize(img, (384,384), interpolation = cv2.INTER_AREA)
        image2[item] = img
    else:
        print(item)

In [10]:
train_df = cd[cd['set'] == 'train'] 
val_df = cd[cd['set'] == 'val'] 
test_df = cd[cd['set'] == 'test'] 

train_graph1 = [image1[i] for i in train_df['file']]
val_graph1 = [image1[i] for i in val_df['file']]
test_graph1 = [image1[i] for i in test_df['file']]
all_graph1 = [image1[i] for i in cd['file']]

train_graph2 = [image2[i] for i in train_df['file']]
val_graph2 = [image2[i] for i in val_df['file']]
test_graph2 = [image2[i] for i in test_df['file']]
all_graph2 = [image2[i] for i in cd['file']]

x_train1 = np.array(train_graph1)/255
x_val1 = np.array(val_graph1)/255
x_test1 = np.array(test_graph1)/255
x_all1 = np.array(all_graph1)/255

x_train2 = np.array(train_graph2)/255
x_val2 = np.array(val_graph2)/255
x_test2 = np.array(test_graph2)/255
x_all2 = np.array(all_graph2)/255

train_cost = np.array(train_df['Cd'])
val_cost = np.array(val_df['Cd'])
test_cost = np.array(test_df['Cd'])
all_cost = np.array(cd['Cd'])

image_shape = train_graph1[0].shape

In [ ]:
test_df = cd[cd['set'] == 'test'] 
test_graph1 = [image1[i] for i in test_df['file']]
test_graph2 = [image2[i] for i in test_df['file']]
x_test1 = np.array(test_graph1)/255
x_test2 = np.array(test_graph2)/255
test_cost = np.array(test_df['Cd'])

x_test1 = torch.tensor(x_test1,dtype=torch.float)
x_test1 = torch.transpose(x_test1, 1, 3)
x_test2 = torch.tensor(x_test2,dtype=torch.float)
x_test2 = torch.transpose(x_test2, 1, 3)
test_cost = torch.tensor(test_cost,dtype=torch.float)

In [12]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 24

x_train1 = torch.tensor(x_train1,dtype=torch.float)
x_train1 = torch.transpose(x_train1, 1, 3)
x_train2 = torch.tensor(x_train2,dtype=torch.float)
x_train2 = torch.transpose(x_train2, 1, 3)
train_cost = torch.tensor(train_cost,dtype=torch.float)

x_val1 = torch.tensor(x_val1,dtype=torch.float)
x_val1 = torch.transpose(x_val1, 1, 3)
x_val2 = torch.tensor(x_val2,dtype=torch.float)
x_val2 = torch.transpose(x_val2, 1, 3)
val_cost = torch.tensor(val_cost,dtype=torch.float)

x_test1 = torch.tensor(x_test1,dtype=torch.float)
x_test1 = torch.transpose(x_test1, 1, 3)
x_test2 = torch.tensor(x_test2,dtype=torch.float)
x_test2 = torch.transpose(x_test2, 1, 3)
test_cost = torch.tensor(test_cost,dtype=torch.float)

x_all1 = torch.tensor(x_all1,dtype=torch.float)
x_all1 = torch.transpose(x_all1, 1, 3)
x_all2 = torch.tensor(x_all2,dtype=torch.float)
x_all2 = torch.transpose(x_all2, 1, 3)
all_cost = torch.tensor(all_cost,dtype=torch.float)

train_data = TensorDataset(x_train1, x_train2, train_cost)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

val_data = TensorDataset(x_val1, x_val2, val_cost)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

test_data = TensorDataset(x_test1, x_test2, test_cost)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=batch_size)

all_data = TensorDataset(x_all1, x_all2, all_cost)
all_sampler = SequentialSampler(all_data)
all_dataloader = DataLoader(all_data, sampler = all_sampler, batch_size=batch_size)

In [13]:
import torchvision.models as models
from attention import ScaledDotProductAttention
pretrained01 = models.resnext101_32x8d(pretrained=True)
pretrained = pretrained01.to(device)
pretrained = nn.Sequential(*list(pretrained.children())[:-2])

pretrained02 = models.resnext101_32x8d(pretrained=True)
pretrained1 = pretrained02.to(device)
pretrained1 = nn.Sequential(*list(pretrained1.children())[:-2])

# from torchsummary import summary
# summary(model2, (3,384,384))

In [14]:
fc = 128
drop1 = 0.0
drop2 = 0.0
class Reshape(nn.Module):
    def __init__(self, *args):
        super(Reshape, self).__init__()
        self.shape = args
    def forward(self, x):
        return x.reshape([x.shape[0],x.shape[1],x.shape[2]*x.shape[3]])
class NORMAL(nn.Module):

    def __init__(self, pretrained):
      
        super(NORMAL, self).__init__()
        self.pretrained = pretrained
        self.attn = ScaledDotProductAttention(dim=fc)
        self.flat = nn.Flatten()
        self.reshape = Reshape()
        self.dropout1 = nn.Dropout(drop1)
        self.dropout2 = nn.Dropout(drop2)
        self.relu =  nn.ReLU()
        self.fc = fc
        self.fc1 = nn.Linear(2048,self.fc)
        self.fc2 = nn.Linear(2048,self.fc)
        self.fc3 = nn.Linear(2048,self.fc)
        self.fc4 = nn.Linear(144*self.fc,self.fc)
        self.fc5 = nn.Linear(self.fc,1)

    def forward(self, x_train):
        x = self.pretrained(x_train)
        y = self.reshape(x)
        y = torch.permute(y, (0, 2, 1))
        key = self.fc2(y)
        key = self.relu(key)
        query = self.fc2(y)
        query = self.relu(query)
        value = self.fc3(y)
        value = self.relu(value)
        context, attn = self.attn(query, key, value)
        context = self.flat(context)
        context = self.dropout1(context)
        context = self.fc4(context)
        context = self.relu(context)
        x = self.fc5(context)
        x = self.relu(x)
        return x, context, key, query, value
    
normal = NORMAL(pretrained)
path1 = 'trained models/attn2_rec_normal_resnext_8e-05_128_0.96_large_3.pt'
normal.load_state_dict(torch.load(path1))
# for param in normal.parameters():
#     param.requires_grad = False

<All keys matched successfully>

In [16]:
class DEPTH(nn.Module):
    def __init__(self, pretrained1):    
        super(DEPTH, self).__init__()
        self.pretrained = pretrained1
        self.attn = ScaledDotProductAttention(dim=fc)
        self.flat = nn.Flatten()
        self.reshape = Reshape()
        self.relu =  nn.ReLU()
        self.fc = fc
        self.fc1 = nn.Linear(2048,self.fc)
        self.fc2 = nn.Linear(2048,self.fc)
        self.fc3 = nn.Linear(144*self.fc,self.fc)
        self.out = nn.Linear(self.fc,1)
    def forward(self, x_train):
        x = self.pretrained(x_train)
        y = self.reshape(x)
        y = torch.permute(y, (0, 2, 1))
        key = self.fc1(y)
        key = self.relu(key)
        query = self.fc1(y)
        query = self.relu(query)
        value = self.fc2(y)
        value = self.relu(value)
        context, attn = self.attn(query, key, value)
        context = self.flat(context)
        context = self.fc3(context)
        context = self.relu(context)
        x = self.out(context)
        x = self.relu(x)
        return x, context, key, query, value
    
depth = DEPTH(pretrained1)
path2 = 'trained models/attn2_rec_depth_resnext_3e-05_128_0.96_large_3.pt'
depth.load_state_dict(torch.load(path2))
for param in depth.parameters():
    param.requires_grad = False

# simple

In [17]:
# class CNN(nn.Module):
#     def __init__(self, normal, depth):
      
#         super(CNN, self).__init__()
#         self.model1 = normal
#         self.model2 = depth
#         self.out = nn.Linear(2,1)
#         with torch.no_grad():
#             self.out.weight = nn.Parameter(torch.tensor(reg.coef_, dtype=torch.float).reshape(1,2))
# #             self.out.weight = nn.Parameter(torch.tensor([1,0], dtype=torch.float).reshape(1,2))
#             self.out.bias = nn.Parameter(torch.tensor(np.array([bias]), dtype=torch.float))
#         self.relu = nn.ReLU()
#     def forward(self, x_train1, x_train2):
#         x1,context1,key1,query1,value1 = self.model1(x_train1)
#         x2,context2,key2,query2,value2 = self.model2(x_train2)  
#         z = torch.cat([x, y],dim=1)
#         z = self.out(z)
#         z = self.relu(z)
#         return z, x1, x2
    # model = CNN(normal, depth)

In [18]:
# print (list(depth.children()))

In [19]:
# len(list(depth.children()))

# cross-attention

In [20]:
class CNN(nn.Module):
    def __init__(self, normal, depth):
        super(CNN, self).__init__()
        super(CNN, self).__init__()
        self.model1 = normal
        self.model2 = depth
        self.fc = fc
        self.relu =  nn.ReLU()
        self.attn = ScaledDotProductAttention(dim=fc)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(165*self.fc,self.fc)
        self.fc2 = nn.Linear(165*self.fc,self.fc)
        self.out = nn.Linear(self.fc*2,1)
    def forward(self, x_train1, x_train2):
        x1,context1,key1,query1,value1 = self.model1(x_train1)
        x2,context2,key2,query2,value2 = self.model2(x_train2)        
        context12, attn12 = self.attn(query1, key2, value2)
        context12 = self.flat(context12)
        context12 = self.fc1(context12)
        context12 = self.relu(context12)
        context21, attn21 = self.attn(query2, key1, value1)
        context21 = self.flat(context21)
        context21 = self.fc2(context21)
        context21 = self.relu(context21)
        x = torch.cat([context12, context21],dim=1)  
        x = self.out(x)
        x = self.relu(x)
        return x, context12, context12

crossattn = CNN(normal, depth)
path = 'trained models/attn_cross_6e-05_0.96_attn_3.pt'
crossattn.load_state_dict(torch.load(path))
crossattn = crossattn.to(device)
for param in crossattn.parameters():
    param.requires_grad = False

# simple fuse

In [21]:
out_1 = nn.Sequential(*list(normal.children())[11:12])
out_2 = nn.Sequential(*list(depth.children())[8:9])
out = torch.cat([out_1[0].weight*0.95, out_2[0].weight*0.05], dim=1)
class CNN(nn.Module):
    def __init__(self, normal, depth):  #drop_3, l_1_3, l_2_3, l_3_3
        super(CNN, self).__init__()
        self.model1 = normal
        self.model2 = depth
        self.fc = fc
        self.relu =  nn.ReLU()
        self.out = nn.Linear(2*self.fc,1)
        with torch.no_grad():
            self.out.weight = nn.Parameter(out)
            self.out.bias = nn.Parameter(torch.tensor(np.array([0]), dtype=torch.float))
    def forward(self, x_train1, x_train2):
        x1,context1,key1,query1,value1 = self.model1(x_train1)
        x2,context2,key2,query2,value2 = self.model2(x_train2) 
        x = torch.cat([context1, context2],dim=1)
        x = self.out(x)
        x = self.relu(x)
        return x, context1, context2
model = CNN(normal, depth)

simfuse = CNN(normal, depth)
path = 'trained models/attn_simple_5e-05_0.96_attn_1.pt'
simfuse.load_state_dict(torch.load(path))
simfuse = simfuse.to(device)
# for param in simfuse.parameters():
#     param.requires_grad = False

# attention fuse

In [22]:
out1 = list(simfuse.children())[3:4]
out2 = list(crossattn.children())[7:8]
out = torch.cat([out1[0].weight*0.99, out2[0].weight*0.01], dim=1)
bias1 = out1[0].bias + out2[0].bias
class CNN(nn.Module):
    def __init__(self, simfuse, crossattn): 
        super(CNN, self).__init__()
        self.model1 = simfuse
        self.model2 = crossattn
        self.fc = fc
        self.relu =  nn.ReLU()
        self.out = nn.Linear(4*self.fc,1)
        with torch.no_grad():
            self.out.weight = nn.Parameter(out)
            self.out.bias = nn.Parameter(bias1)
    def forward(self, x_train1, x_train2):
        x1,context1,context2 = self.model1(x_train1,x_train2)
        x2,context3,context4 = self.model2(x_train1,x_train2) 
        x = torch.cat([context1, context2, context3, context4],dim=1)
        x = self.out(x)
        x = self.relu(x)
        return x, x1, x2
model = CNN(simfuse, crossattn)

# multimodal = CNN(normal, depth)
# path = 'trained models/attn+_img_7e-05_0.96_fuse-best.pt'
# multimodal.load_state_dict(torch.load(path))

In [23]:
# out1 = list(simfuse.children())[3:4]
# out2 = list(crossattn.children())[7:8]
# out = torch.cat([out1[0].weight*0.99, out2[0].weight*0.01], dim=1)
# bias1 = out1[0].bias + out2[0].bias
# class CNN(nn.Module):
#     def __init__(self, normal, depth, crossattn): 
#         super(CNN, self).__init__()
#         self.model1 = normal
#         self.model3 = depth
#         self.model2 = crossattn
#         self.fc = fc
#         self.relu =  nn.ReLU()
#         self.out = nn.Linear(4*self.fc,1)
#         with torch.no_grad():
#             self.out.weight = nn.Parameter(out)
#             self.out.bias = nn.Parameter(bias1)
#     def forward(self, x_train1, x_train2):
#         x1, context1, key1, query1, value1 = self.model1(x_train1)
#         x2,context3,context4 = self.model2(x_train1,x_train2) 
#         x3, context2, key2, query2, value2 = self.model3(x_train2)
#         x = torch.cat([context1, context2, context3, context4],dim=1)
#         x = self.out(x)
#         x = self.relu(x)
#         return x, x1, x2
# model = CNN(normal, depth, crossattn)

# # multimodal = CNN(normal, depth)
# # path = 'trained models/attn+_img_7e-05_0.96_fuse-best.pt'
# # multimodal.load_state_dict(torch.load(path))

In [20]:
# with torch.no_grad():
# #     for layer in model.state_dict():
# #         print(layer)
#         #print(torch.ones_like(mask_model.state_dict()[layer].data))
# #         mask_model.state_dict()[layer].data.fill_(1)
#     model.state_dict()['out.bias'].data.fill_(bias)

# for param in model.parameters():
#     param.requires_grad = False

In [24]:
# push the model to GP
model = model.to(device)

b = 0
num_warmup_steps = len(train_dataloader) * b

# optimizer from hugging face transformers
import torch.optim as optim
from ignite.handlers import param_scheduler

lr = 30e-6
decay = 0.96
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decay)

In [26]:
# define the loss function
mse  = nn.MSELoss() 
# metric = nn.RootMeanSquaredError()

# function to train the model
def train():
  
    model.train()

    total_loss, total_rmse = 0, 0
  
    # empty list to save model predictions
    total_preds=[]
  
    # iterate over batches
    for step,batch in enumerate(train_dataloader):
    
        # progress update after every 50 batches.
        if step % 20 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]

        x_train1, x_train2, train_cost = batch
#         print (x_train.shape)

        # clear previously calculated gradients 
        model.zero_grad()        

        # get model predictions for the current batch
        preds,c1,c2 = model(x_train1, x_train2)
#         print (preds[:,0].shape, train_cost.shape)

        # compute the loss between actual and predicted values
        loss = mse(preds[:,0], train_cost)

        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

        # append the model predictions
        total_preds.append(preds)
        
    scheduler.step()

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)
  
    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    #returns the loss and predictions
    return avg_loss, total_preds

In [27]:
# function for evaluating the model

def evaluate():
  
    print("\nEvaluating...")
  
    # deactivate dropout layers
    model.eval()

    total_loss, total_rmse = 0, 0
  
    # empty list to save the model predictions
    total_preds = []

    # iterate over batches
    for step,batch in enumerate(val_dataloader):
    
        # Progress update every 50 batches.
        if step % 20 == 0 and not step == 0:

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        x_val1, x_val2, val_cost = batch

        # deactivate autograd
        with torch.no_grad():

            # model predictions
            preds,c1,c2 = model(x_val1, x_val2)

            # compute the validation loss between actual and predicted values
            loss = mse(preds[:,0],val_cost)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()

            total_preds.append(preds)

    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader) 

    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    return avg_loss, total_preds

In [28]:
def test(test_dataloader):
  
    print("\nEvaluating...")
  
    # deactivate dropout layers
    model.eval()

    total_loss, total_rmse = 0, 0
  
    # empty list to save the model predictions
    total_preds = []

    # iterate over batches
    for step,batch in enumerate(test_dataloader):
    
        # Progress update every 50 batches.
        if step % 20 == 0 and not step == 0:

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(test_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        x_val1, x_val2, val_cost = batch

        # deactivate autograd
        with torch.no_grad():

            # model predictions
            preds,c1,c2 = model(x_val1, x_val2)

            preds = preds.detach().cpu().numpy()

            total_preds.append(preds)

    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)

    return total_preds

In [26]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

# number of training epochs
epochs = 1000

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 
                   'trained models/attn+_fuse_{}_{}_5.pt'.format(lr,decay))
        i = 0
    else:
        i += 1
    if i == 3:
        break
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.5f}')
    print(f'Validation Loss: {valid_loss:.5f}')


 Epoch 1 / 1000
  Batch    20  of    265.
  Batch    40  of    265.
  Batch    60  of    265.
  Batch    80  of    265.
  Batch   100  of    265.
  Batch   120  of    265.
  Batch   140  of    265.
  Batch   160  of    265.
  Batch   180  of    265.
  Batch   200  of    265.
  Batch   220  of    265.
  Batch   240  of    265.
  Batch   260  of    265.

Evaluating...
  Batch    20  of     57.
  Batch    40  of     57.

Training Loss: 0.00013
Validation Loss: 0.00164

 Epoch 2 / 1000
  Batch    20  of    265.
  Batch    40  of    265.
  Batch    60  of    265.
  Batch    80  of    265.
  Batch   100  of    265.
  Batch   120  of    265.
  Batch   140  of    265.
  Batch   160  of    265.
  Batch   180  of    265.
  Batch   200  of    265.
  Batch   220  of    265.
  Batch   240  of    265.
  Batch   260  of    265.

Evaluating...
  Batch    20  of     57.
  Batch    40  of     57.

Training Loss: 0.00007
Validation Loss: 0.00167

 Epoch 3 / 1000
  Batch    20  of    265.
  Batch    40  

KeyboardInterrupt: 

In [61]:
test_df = cd[cd['set'] == 'test']
test_dfx = test_df[test_df['Cd'] > 0.8]
test_graph1 = [image1[i] for i in test_dfx['file']]
test_graph2 = [image2[i] for i in test_dfx['file']]
x_test1 = np.array(test_graph1)/255
x_test2 = np.array(test_graph2)/255
test_cost = np.array(test_dfx['Cd'])

x_test1 = torch.tensor(x_test1,dtype=torch.float)
x_test1 = torch.transpose(x_test1, 1, 3)
x_test2 = torch.tensor(x_test2,dtype=torch.float)
x_test2 = torch.transpose(x_test2, 1, 3)
test_cost = torch.tensor(test_cost,dtype=torch.float)

test_data = TensorDataset(x_test1, x_test2, test_cost)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, batch_size=batch_size)

In [62]:
# ##Make Predictions
# #load weights of best model
path = 'trained models/attn+_fuse_{}_{}_5.pt'.format(lr,decay)
# # # path = 'trained models/side_resnext_200_8e-05_128_0.5.pt'
# path = 'trained models/attn+_img_7e-05_0.96_fuse-best.pt'
model.load_state_dict(torch.load(path))

# get predictions for test data
test_result = test(test_dataloader)[:,0]

difference_array = np.subtract(test_result, test_cost)
squared_array = np.square(difference_array)
mse = squared_array.mean().numpy()

correlation_matrix = np.corrcoef(test_cost, test_result)
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

idx = 0
var = 'integrated'
output = pd.DataFrame()
# output.loc[idx,'warm up'] = a
output.loc[idx,'var'] = var
# output.loc[idx,'lr'] = lr
output.loc[idx,'mse'] = mse
output.loc[idx,'r^2'] = r_squared
output.loc[idx,'fc'] = fc
output.loc[idx,'lr'] = lr
output.loc[idx,'decay'] = decay
output.loc[idx,'tolerance'] = 20
output.loc[idx,'diff'] = abs(difference_array).mean().numpy()


Evaluating...


C:\Users\sbyli\anaconda3\envs\kaolin\lib\site-packages\numpy\lib\function_base.py:2691: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\sbyli\anaconda3\envs\kaolin\lib\site-packages\numpy\lib\function_base.py:2692: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [63]:
output

,var,mse,r^2,fc,lr,decay,tolerance,diff
0,integrated,0.047334,NaN,128.0,0.00003,0.96,20.0,0.217565


In [1]:
plot = pd.DataFrame()
plot['Cd'] = test_cost
plot['Predicted'] = test_result
plot = plot.sort_values(by=['Cd'], ascending = True, na_position='first').reset_index(drop = True)
    
correlation_matrix = np.corrcoef(plot['Cd'], plot['Predicted'])
correlation_xy = correlation_matrix[0,1]
r_squared = correlation_xy**2

from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
figure(figsize=(4, 3), dpi=400)
plt.scatter(plot['Cd'], plot['Predicted'], s = 5)

x = [0,1]
y = [0,1]
plt.plot(x,y, color="black")

plt.xlim([0,1])
 plt.ylim([0,1])

plt.xlabel('Ground Truth')
plt.ylabel('Predicted')
plt.tight_layout()
# plt.savefig('attn+_fuset.png'.format(lr,decay),dpi=400)
print (var, r_squared)